## Test pipeline to convert SVG to DST(csv) file 

Preprocess the dataset:
1. Convert all svg tags to path tags
2. convert all curves and to cubic bezier curves
3. Train a model to predict the DST file from the SVG file

Preprocess the dataset: done separately for now

In [1]:
import pickle
from tqdm import tqdm
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine the computing device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# File paths for data
svg_paths_file = "/kaggle/input/initial-data/svg_paths.pkl"
stitches_file = "/kaggle/input/initial-data/stitches.pkl"

In [4]:
# Load SVG paths and stitches data
with open(svg_paths_file, 'rb') as file:
    svg_paths = pickle.load(file)

with open(stitches_file, 'rb') as file:
    stitches = pickle.load(file)

In [5]:
# Load BART tokenizer and model, specify device directly in the model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large', device_map='auto',use_cache=True)
#model = torch.nn.DataParallel(model)  # DataParallel usage
#model.to(device)  # Ensure model is sent to device

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [6]:
# Setup for training
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 30
batch_size = 1

In [7]:
import os
# Set CUDA_LAUNCH_BLOCKING to 1 for detailed debugging
os.environ['CUDA_LAUNCH_BLOCKING']="1"

def check_inputs(tensor, name="Tensor"):
    if torch.isnan(tensor).any():
        print(f"NaN found in {name}")
    if torch.isinf(tensor).any():
        print(f"Inf found in {name}")

In [8]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0

    # Processing batches
    for i in range(0, len(svg_paths), batch_size):
        batch_svg_paths = svg_paths[i:i+batch_size]
        batch_stitches = stitches[i:i+batch_size]

        # Prepare inputs and labels with padding and attention mask
        inputs = tokenizer(batch_svg_paths, return_tensors='pt', padding=True, truncation=True)
        labels = tokenizer(batch_stitches, return_tensors='pt', padding=True, truncation=True)
        
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)
        labels = labels.input_ids.to(device)

        # Forward and backward passes
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate average loss
    average_loss = total_loss / len(svg_paths)
    print(f"Epoch {epoch+1}: Average Loss = {average_loss}")

  0%|          | 0/30 [00:00<?, ?it/s]2024-05-13 11:04:29.983656: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 11:04:29.983754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 11:04:30.090127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
  3%|▎         | 1/30 [02:01<58:37, 121.28s/it]

Epoch 1: Average Loss = 1.8375477259702022


  7%|▋         | 2/30 [03:56<54:50, 117.53s/it]

Epoch 2: Average Loss = 1.2699277542605258


 10%|█         | 3/30 [05:50<52:19, 116.28s/it]

...

Epoch 28: Average Loss = 0.6278668866299166


 97%|█████████▋| 29/30 [55:26<01:54, 114.40s/it]

Epoch 29: Average Loss = 0.6141275683842083


100%|██████████| 30/30 [57:21<00:00, 114.70s/it]

Epoch 30: Average Loss = 0.7101094433576753


In [18]:
# torch.save(model,"/kaggle/working/bartV5.pth")
torch.save(model.state_dict(), '/kaggle/working/bartV6.pth')

In [14]:
torch.cuda.empty_cache()
torch.cuda.synchronize()

Testing the model

In [1]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
# Set CUDA_LAUNCH_BLOCKING to 1 for detailed debugging
os.environ['CUDA_LAUNCH_BLOCKING']="1"

def check_inputs(tensor, name="Tensor"):
    if torch.isnan(tensor).any():
        print(f"NaN found in {name}")
    if torch.isinf(tensor).any():
        print(f"Inf found in {name}")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
modelTest = BartForConditionalGeneration.from_pretrained('facebook/bart-large', device_map='auto',use_cache=True)
# modelTest.load_state_dict("/kaggle/working/bartV6.pth")
# Load the saved state dictionary
state_dict = torch.load("/kaggle/working/bartV6.pth")

# Load the state dictionary into the model
modelTest.load_state_dict(state_dict)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
test_input = "M 0.06 12.57 C 0.06 12.57 30.62 12.57 30.62 12.57 C 30.62 12.57 0.07 63.25 0.07 63.25 C 0.07 63.25 0 63.37 0 63.37 C 0 63.37 0.07 74.2 0.07 74.2 C 0.07 74.2 43.9 74.2 43.9 74.2 C 43.9 74.2 43.9 63.04 43.9 63.04 C 43.9 63.04 13.02 63.04 13.02 63.04 C 13.02 63.04 43.9 12.21 43.9 12.21 C 43.9 12.21 43.9 1.41 43.9 1.41 C 43.9 1.41 0.06 1.41 0.06 1.41 C 0.06 1.41 0.06 12.57 0.06 12.57 C 0.06 12.57 0.06 12.57 0.06 12.57 M 54.2 74.2 C 54.2 74.2 94.62 74.2 94.62 74.2 C 94.62 74.2 94.62 63.04 94.62 63.04 C 94.62 63.04 65.36 63.04 65.36 63.04 C 65.36 63.04 65.36 43.34 65.36 43.34 C 65.36 43.34 91.1 43.34 91.1 43.34 C 91.1 43.34 91.1 32.17 91.1 32.17 C 91.1 32.17 65.36 32.17 65.36 32.17 C 65.36 32.17 65.36 12.57 65.36 12.57 C 65.36 12.57 94.62 12.57 94.62 12.57 C 94.62 12.57 94.62 1.41 94.62 1.41 C 94.62 1.41 54.2 1.41 54.2 1.41 C 54.2 1.41 54.2 74.2 54.2 74.2 C 54.2 74.2 54.2 74.2 54.2 74.2 M 129.28 1.41 C 129.28 1.41 104.14 1.41 104.14 1.41 C 104.14 1.41 104.14 74.2 104.14 74.2 C 104.14 74.2 115.3 74.2 115.3 74.2 C 115.3 74.2 115.3 12.57 115.3 12.57 C 115.3 12.57 129.27 12.57 129.27 12.57 C 132.15 12.57 135.01000000000002 13.8 137.79000000000002 16.22 C 140.54000000000002 18.619999999999997 141.94000000000003 21.83 141.94000000000003 25.74 C 141.94000000000003 30.049999999999997 140.52000000000004 33.35 137.71000000000004 35.55 C 134.86000000000004 37.779999999999994 132.47000000000003 38.91 130.58000000000004 38.91 C 130.58000000000004 38.91 117.29000000000005 39.019999999999996 117.29000000000005 39.019999999999996 C 117.29000000000005 39.019999999999996 135.40000000000003 74.19999999999999 135.40000000000003 74.19999999999999 C 135.40000000000003 74.19999999999999 148.19000000000003 74.19999999999999 148.19000000000003 74.19999999999999 C 148.19000000000003 74.19999999999999 134.94000000000003 48.89999999999999 134.94000000000003 48.89999999999999 C 140.76000000000002 47.419999999999995 145.23000000000002 44.54999999999999 148.23000000000002 40.35999999999999 C 151.33 36.03999999999999 152.9 31.11999999999999 152.9 25.739999999999995 C 152.9 18.999999999999993 150.52 13.189999999999994 145.82 8.489999999999995 C 141.12 3.7899999999999947 135.54999999999998 1.399999999999995 129.28 1.399999999999995 C 129.28 1.399999999999995 129.28 1.41 129.28 1.41 M 221.18 11.11 C 213.81 3.7399999999999993 204.86 0 194.58 0 C 184.3 0 175.25 3.74 167.88000000000002 11.11 C 160.51000000000002 18.48 156.77000000000004 27.46 156.77000000000004 37.81 C 156.77000000000004 48.160000000000004 160.51000000000005 57.14 167.88000000000005 64.51 C 175.25000000000006 71.88000000000001 184.23000000000005 75.62 194.58000000000004 75.62 C 204.93000000000004 75.62 213.89000000000004 71.87 221.23000000000005 64.46000000000001 C 228.57000000000005 57.06000000000001 232.28000000000006 48.09 232.28000000000006 37.81000000000001 C 232.28000000000006 27.530000000000015 228.54000000000005 18.48000000000001 221.17000000000007 11.11000000000001 C 221.17000000000007 11.11000000000001 221.18 11.11 221.18 11.11 M 194.59 64.45 C 187.28 64.45 180.95 61.81 175.76 56.61 C 170.57 51.41 167.94 45.08 167.94 37.81 C 167.94 30.540000000000006 170.57 24.220000000000002 175.76 19.01 C 180.95 13.8 187.28 11.170000000000002 194.58999999999997 11.170000000000002 C 201.89999999999995 11.170000000000002 208.12999999999997 13.810000000000002 213.30999999999997 19.01 C 218.49999999999997 24.220000000000002 221.12999999999997 30.54 221.12999999999997 37.81 C 221.12999999999997 45.080000000000005 218.49999999999997 51.400000000000006 213.30999999999997 56.61 C 208.11999999999998 61.81999999999999 201.81999999999996 64.45 194.58999999999997 64.45 C 194.58999999999997 64.45 194.59 64.45 194.59 64.45 "

In [6]:
inputs = tokenizer([test_input], max_length=2048, return_tensors="pt", padding=True, truncation=True).to(device)

In [7]:
output = modelTest.generate(inputs["input_ids"], num_beams=2, min_length=0, early_stopping=False)

/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [22,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [22,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [22,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [22,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [22,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [22,0,0], thread: [69,0,0] Assertion `srcIn

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
